In [16]:
# Import netket library
import netket as nk
from qiskit.chemistry import FermionicOperator
import itertools

# Helper libraries
import numpy as np
import os
import time
import json

In [17]:
n_electrons=2

#H2
OB = np.load('../data/integrals/STO-3G/STO-3G_H2_OB_d0-7339_eq1.npy')
TB = np.load('../data/integrals/STO-3G/STO-3G_H2_TB_d0-7339_eq1.npy')   

FerOp = FermionicOperator(OB, TB)

mapping = FerOp.mapping('jordan_wigner')
weights = [w[0] for w in mapping.paulis]
operators = [w[1].to_label() for w in mapping.paulis]

ha = nk.operator.PauliStrings(operators, weights)
g = nk.graph.Hypercube(n_dim=1, length=n_electrons, pbc=False)
hi = nk.hilbert.Spin(graph=g, s=0.5) #want ha.hilbert werkt niet voor Rbm... ;(
print(hi.size)

# seed
np.random.seed(seed=123)
nk.random.seed(seed=123)

ma = nk.machine.RbmSpin(hi, alpha=2)
ma.init_random_parameters(seed=1234, sigma=0.01)

sa = nk.sampler.MetropolisExchange(machine=ma, n_chains=1)

2


In [14]:
n_down = 0
ten = 0
while n_down != int(n_electrons/2):
    ten+=1
    sa = nk.sampler.MetropolisExchange(machine=ma, n_chains=1)
    for s in sa.samples(1): print(s[0])
    n_down = int(n_electrons - sum(s[0]+1/2))

for s in sa.samples(10):
    print(s)

[-1.  1.]
[[-1.  1.]]
[[-1.  1.]]
[[-1.  1.]]
[[-1.  1.]]
[[-1.  1.]]
[[-1.  1.]]
[[-1.  1.]]
[[-1.  1.]]
[[-1.  1.]]
[[-1.  1.]]


In [15]:
# Optimizer
op = nk.optimizer.Sgd(learning_rate=0.1)

# Stochastic reconfiguration
vmc = nk.variational.Vmc(
    hamiltonian=ha,
    sampler=sa,
    optimizer=op,
    n_samples=2000,
    diag_shift=0.1,
    method='Sr')

for it in vmc.iter(500,50):
    print(it,vmc.energy)

0 (-2 + 0i) ± nan [var=nan, R=nan]


KeyboardInterrupt: 